<a href="https://colab.research.google.com/github/HollyIrvine/mgmt590-HollyIrvine-classResources/blob/main/Team_Assignment_1_Product_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect Colab to BigQuery

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("--- Authenticated to Google Cloud ---")

In [ ]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'mgmt599-hollyirvine-lab1' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
from google.cloud import bigquery
import pandas as pd # Often used to work with query results

# --- Configuration ---
PROJECT_ID = "mgmt599-hollyirvine-lab1"
DATASET_ID = "lab1_eda"
TABLE_ID = "superstore-customer_id2"
FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"--- Connected to BigQuery project: {PROJECT_ID} ---")

## Category Performance

In [ ]:
# prompt: Write a comprehensive query to compare category performance using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Group by category and calculate:
#   - Total revenue (sales)
#   - Total profit
#   - Average profit margin
#   - Total quantity sold
#   - Number of unique products
#   - Number of orders
#   - Average order value
# - Calculate profit-to-sales ratio for each category
# - Include growth metrics if possible (year-over-year comparison using order_date)
# - Order by total profit descending
# - Format monetary values appropriately

import pandas as pd
# SQL query for category performance analysis
query = f"""
WITH CategoryPerformance AS (
    SELECT
        category,
        SUM(sales) AS total_revenue,
        SUM(profit) AS total_profit,
        AVG(profit / sales) AS average_profit_margin,
        SUM(quantity) AS total_quantity_sold,
        COUNT(DISTINCT product_name) AS unique_products,
        COUNT(DISTINCT order_id) AS number_of_orders,
        SUM(sales) / COUNT(DISTINCT order_id) AS average_order_value,
        SUM(profit) / SUM(sales) AS profit_to_sales_ratio,
        EXTRACT(YEAR FROM order_date) AS order_year
    FROM
        `{FULL_TABLE_ID}`
    GROUP BY
        category,
        order_year
)

-- Compare year-over-year performance
SELECT
    cp_current.category,
    cp_current.order_year,
    cp_current.total_revenue,
    cp_current.total_profit,
    cp_current.average_profit_margin,
    cp_current.total_quantity_sold,
    cp_current.unique_products,
    cp_current.number_of_orders,
    cp_current.average_order_value,
    cp_current.profit_to_sales_ratio,
    LAG(cp_current.total_revenue, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year) AS previous_year_revenue,
    (cp_current.total_revenue - LAG(cp_current.total_revenue, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year)) / NULLIF(LAG(cp_current.total_revenue, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year), 0) AS revenue_growth_rate,
    LAG(cp_current.total_profit, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year) AS previous_year_profit,
     (cp_current.total_profit - LAG(cp_current.total_profit, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year)) / NULLIF(LAG(cp_current.total_profit, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year), 0) AS profit_growth_rate
FROM
    CategoryPerformance cp_current
ORDER BY
    cp_current.order_year DESC, -- Show latest year first
    cp_current.total_profit DESC -- Order by total profit descending within each year
"""

# Run the query
query_job = client.query(query)

# Convert results to DataFrame
results_df = query_job.to_dataframe()

# Display the results
print("--- Category Performance Analysis ---")
print(results_df)

# Optional: Format monetary columns for better readability
monetary_cols = ['total_revenue', 'total_profit', 'average_order_value', 'previous_year_revenue', 'previous_year_profit']
for col in monetary_cols:
    if col in results_df.columns:
        results_df[col] = results_df[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else None)

# Optional: Format percentages
percentage_cols = ['average_profit_margin', 'profit_to_sales_ratio', 'revenue_growth_rate', 'profit_growth_rate']
for col in percentage_cols:
     if col in results_df.columns:
        # Handle potential division by zero or zero previous year values gracefully
        results_df[col] = results_df[col].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else None)


print("\n--- Formatted Category Performance Analysis ---")
results_df

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=results_df)

## Product Profitability

In [ ]:
# prompt: analyze product profitability using the table `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Calculate total profit, total sales, average profit margin, and total quantity sold for each product
# - Calculate profit per unit (total profit / total quantity)
# - Rank products by total profit (highest to lowest)
# - Include only products with at least 10 orders to ensure statistical significance
# - Show top 20 most profitable products
# - Include columns: product_name, total_profit, total_sales, avg_profit_margin, total_quantity, profit_per_unit, order_count

import pandas as pd
# SQL query for product profitability analysis
product_query = f"""
SELECT
    product_name,
    SUM(profit) AS total_profit,
    SUM(sales) AS total_sales,
    AVG(profit / sales) AS avg_profit_margin,
    SUM(quantity) AS total_quantity,
    COUNT(DISTINCT order_id) AS order_count
FROM
    `{FULL_TABLE_ID}`
GROUP BY
    product_name
HAVING
    COUNT(DISTINCT order_id) >= 10 -- Filter for products with at least 10 orders
"""

# Run the product profitability query
product_query_job = client.query(product_query)

# Convert product results to DataFrame
product_results_df = product_query_job.to_dataframe()

# Calculate profit per unit
product_results_df['profit_per_unit'] = product_results_df['total_profit'] / product_results_df['total_quantity']

# Rank products by total profit (highest to lowest)
product_results_df = product_results_df.sort_values(by='total_profit', ascending=False)

# Select the top 20 most profitable products
top_20_profitable_products = product_results_df.head(20).copy()

# Display the results
print("\n--- Top 20 Most Profitable Products (with at least 10 orders) ---")
# Optional: Format monetary columns for better readability
monetary_cols_product = ['total_profit', 'total_sales', 'profit_per_unit']
for col in monetary_cols_product:
    if col in top_20_profitable_products.columns:
        top_20_profitable_products[col] = top_20_profitable_products[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else None)

# Optional: Format percentages
percentage_cols_product = ['avg_profit_margin']
for col in percentage_cols_product:
     if col in top_20_profitable_products.columns:
        top_20_profitable_products[col] = top_20_profitable_products[col].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else None)


top_20_profitable_products

## Discount Performance

In [ ]:
# prompt: Write code to analyze the relationship between discounts and product performance using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Create discount bands: 0%, 0-10%, 10-20%, 20-30%, 30%+
# - For each discount band, calculate:
#   - Average profit margin
#   - Total profit
#   - Total sales volume
#   - Average quantity per order
#   - Number of orders
# - Calculate the impact of discount on profit per unit
# - Use CASE statements to categorize discount levels
# - Show which discount ranges are most profitable
# - Include statistical measures like median profit margin per discount band

import pandas as pd
# SQL query to analyze the relationship between discounts and product performance
discount_query = f"""
SELECT
    -- Categorize discount levels using CASE statements
    CASE
        WHEN discount = 0 THEN '0%'
        WHEN discount > 0 AND discount <= 0.1 THEN '0-10%'
        WHEN discount > 0.1 AND discount <= 0.2 THEN '10-20%'
        WHEN discount > 0.2 AND discount <= 0.3 THEN '20-30%'
        WHEN discount > 0.3 THEN '30%+'
        ELSE 'Other' -- Should not happen with the given data, but good for robustness
    END AS discount_band,
    -- Calculate performance metrics for each discount band
    AVG(profit / sales) AS average_profit_margin,
    SUM(profit) AS total_profit,
    SUM(sales) AS total_sales_volume,
    AVG(quantity) AS average_quantity_per_order,
    COUNT(DISTINCT order_id) AS number_of_orders,
    AVG(profit / quantity) AS average_profit_per_unit, -- Impact of discount on profit per unit
    -- Include statistical measures like median profit margin per discount band
    APPROX_QUANTILES(profit / sales, 2)[OFFSET(1)] AS median_profit_margin
FROM
    `{FULL_TABLE_ID}`
GROUP BY
    discount_band
ORDER BY
    -- Order by total profit descending to show most profitable bands first
    total_profit DESC
"""

# Run the discount analysis query
discount_query_job = client.query(discount_query)

# Convert discount analysis results to DataFrame
discount_results_df = discount_query_job.to_dataframe()

# Display the results
print("\n--- Discount Level Performance Analysis ---")

# Optional: Format monetary columns for better readability
monetary_cols_discount = ['total_profit', 'total_sales_volume', 'average_profit_per_unit']
for col in discount_results_df.columns:
    if col in monetary_cols_discount:
        # Convert to numeric first, then format
        discount_results_df[col] = pd.to_numeric(discount_results_df[col], errors='coerce').apply(
            lambda x: f"-${abs(x):,.2f}" if pd.notnull(x) and x < 0
            else f"${x:,.2f}" if pd.notnull(x) else None
        )

# Optional: Format percentages and ratios
percentage_cols_discount = ['average_profit_margin', 'median_profit_margin']
for col in discount_results_df.columns:
    if col in percentage_cols_discount:
        discount_results_df[col] = discount_results_df[col].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else None)

# Format average quantities to integers
discount_results_df['average_quantity_per_order'] = discount_results_df['average_quantity_per_order'].round().astype(int)

discount_results_df

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=discount_results_df)

## Strategic Product Portfolio

In [ ]:
# prompt: 4.	Create a strategic product portfolio matrix using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Calculate for each product:
# -- Total profit (profitability measure)
# -- Total sales volume (market size measure)
# -- Profit growth rate (if comparing time periods)
# -- Market share within category
# - Create a 2x2 matrix classification:
# -- High Profit + High Volume = "Stars"
# -- High Profit + Low Volume = "Cash Cows"
# -- Low Profit + High Volume = "Question Marks"
# -- Low Profit + Low Volume = "Dogs"
# -- Use CASE statements to assign each product to a quadrant
# - Calculate percentiles for profit and volume to determine high/low thresholds
# - Show count of products in each quadrant and their total contribution

import pandas as pd
# SQL query for product portfolio matrix
portfolio_query = f"""
WITH ProductMetrics AS (
    SELECT
        product_name,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales,
        -- You can add growth rate calculations here if needed,
        -- potentially by joining or using window functions on year data.
        -- For simplicity, we'll focus on total profit and total sales volume.
        -- To calculate market share within category, you would need to join with
        -- a table or CTE that has total sales per category.
        -- Example (requires joining with a table that has category info):
        -- SUM(sales) / SUM(sales) OVER (PARTITION BY category) AS market_share_category
        'Placeholder Category' as category -- Replace with actual category if available
    FROM
        `{FULL_TABLE_ID}`
    GROUP BY
        product_name
),
ProfitPercentiles AS (
    SELECT
        product_name,
        total_profit,
        total_sales,
        category,
        NTILE(4) OVER (ORDER BY total_profit) AS profit_quartile
    FROM
        ProductMetrics
),
SalesPercentiles AS (
    SELECT
        product_name,
        total_profit,
        total_sales,
        category,
        profit_quartile,
        NTILE(4) OVER (ORDER BY total_sales) AS sales_quartile
    FROM
        ProfitPercentiles
)
SELECT
    product_name,
    total_profit,
    total_sales,
    category,
    profit_quartile,
    sales_quartile,
    CASE
        WHEN profit_quartile >= 3 AND sales_quartile >= 3 THEN 'Stars' -- High Profit, High Volume (Using >= 3 for top 50%)
        WHEN profit_quartile >= 3 AND sales_quartile < 3 THEN 'Cash Cows' -- High Profit, Low Volume
        WHEN profit_quartile < 3 AND sales_quartile >= 3 THEN 'Question Marks' -- Low Profit, High Volume
        WHEN profit_quartile < 3 AND sales_quartile < 3 THEN 'Dogs' -- Low Profit, Low Volume
        ELSE 'Other'
    END AS portfolio_quadrant
FROM
    SalesPercentiles
"""

# Run the portfolio analysis query
portfolio_query_job = client.query(portfolio_query)

# Convert portfolio analysis results to DataFrame
portfolio_results_df = portfolio_query_job.to_dataframe()

# Display the results
print("\n--- Strategic Product Portfolio Analysis ---")
portfolio_results_df

# Calculate counts and total contribution per quadrant
quadrant_summary = portfolio_results_df.groupby('portfolio_quadrant').agg(
    product_count=('product_name', 'count'),
    total_profit_contribution=('total_profit', 'sum'),
    total_sales_contribution=('total_sales', 'sum')
).reset_index()

# Display the quadrant summary
print("\n--- Product Portfolio Quadrant Summary ---")
quadrant_summary

# Optional: Format summary monetary columns for better readability
monetary_cols_summary = ['total_profit_contribution', 'total_sales_contribution']
for col in quadrant_summary.columns:
    if col in monetary_cols_summary:
        quadrant_summary[col] = quadrant_summary[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else None)

quadrant_summary


## Profitability Drivers

In [ ]:
# prompt: 5.	Create a query to identify key drivers of product profitability using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Group by product_name and calculate:
# -- Average discount rate
# -- Average profit margin
# -- Total profit
# -- Average order value (sales/quantity)
# - Calculate correlation indicators between discount and profitability
# - Show products with highest profit margins vs. lowest discount rates
# - Include a profitability_score calculation: (total_profit * avg_profit_margin) / (1 + avg_discount)
# - Order by profitability_score descending - Limit to top 25 products

def analyze_product_profitability(client, table_id):
    """Analyze product profitability drivers with categories."""

    # Query including category
    query = f"""
    SELECT
        product_name,
        AVG(discount) AS avg_discount_rate,
        AVG(profit / sales) AS avg_profit_margin,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales,
        SUM(sales) / COUNT(DISTINCT order_id) AS avg_order_value_per_order,
        (SUM(profit) * AVG(profit / sales)) / (1 + AVG(discount)) AS profitability_score
    FROM
        `{table_id}`
    GROUP BY
        product_name
    ORDER BY
        profitability_score DESC
    LIMIT 25
    """

    # Execute and format
    df = client.query(query).to_dataframe()

    # Create formatted copy
    df_display = df.copy()

    # Apply formatting
    format_rules = {
        'total_profit': '${:,.2f}',
        'avg_order_value_per_order': '${:,.2f}',
        'profitability_score': '${:,.2f}',
        'avg_discount_rate': '{:.2%}',
        'avg_profit_margin': '{:.2%}'
    }

    for col, fmt in format_rules.items():
        if col in df_display.columns:
            df_display[col] = df[col].apply(lambda x: fmt.format(x) if pd.notnull(x) else None)

    # Calculate correlations
    numeric_cols = ['avg_discount_rate', 'avg_profit_margin', 'total_profit',
                    'avg_order_value_per_order', 'profitability_score']
    correlations = df[numeric_cols].corr()

    # High margin products
    high_margin_products = df.nlargest(10, 'avg_profit_margin')

    return df_display, correlations, high_margin_products

# Usage
products_df, corr_matrix, high_margin = analyze_product_profitability(client, FULL_TABLE_ID)

print("\n--- Product Profitability Analysis ---")
print(products_df)
print("\n--- Correlations ---")
print(corr_matrix)

In [ ]:
# prompt: 14.	Create a strategic product portfolio matrix, divided by region, using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Calculate for each product:
# -- Total profit (profitability measure)
# -- Total sales volume (market size measure)
# -- Profit growth rate (if comparing time periods)
# -- Market share within category
# For each region:
# - Create a 2x2 matrix classification:
# -- High Profit + High Volume = "Stars"
# -- High Profit + Low Volume = "Cash Cows"
# -- Low Profit + High Volume = "Question Marks"
# -- Low Profit + Low Volume = "Dogs"
# -- Use CASE statements to assign each product to a quadrant
# - Calculate percentiles for profit and volume to determine high/low thresholds
# - Show count of products in each quadrant and their total contribution

import pandas as pd
# SQL query for strategic product portfolio matrix by region
portfolio_query_region = f"""
WITH RegionalProductMetrics AS (
    SELECT
        region,
        product_name,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales,
        -- Add product category here if available in the original table
        -- category -- Example: category field from the original table
    FROM
        `{FULL_TABLE_ID}`
    GROUP BY
        region,
        product_name
),
RegionalProfitPercentiles AS (
    SELECT
        region,
        product_name,
        total_profit,
        total_sales,
        -- category, -- Example: category field
        NTILE(4) OVER (PARTITION BY region ORDER BY total_profit) AS profit_quartile_region
    FROM
        RegionalProductMetrics
),
RegionalSalesPercentiles AS (
    SELECT
        region,
        product_name,
        total_profit,
        total_sales,
        -- category, -- Example: category field
        profit_quartile_region,
        NTILE(4) OVER (PARTITION BY region ORDER BY total_sales) AS sales_quartile_region
    FROM
        RegionalProfitPercentiles
)
SELECT
    region,
    product_name,
    total_profit,
    total_sales,
    -- category, -- Example: category field
    profit_quartile_region,
    sales_quartile_region,
    CASE
        WHEN profit_quartile_region >= 3 AND sales_quartile_region >= 3 THEN 'Stars' -- High Profit, High Volume (Using >= 3 for top 50%)
        WHEN profit_quartile_region >= 3 AND sales_quartile_region < 3 THEN 'Cash Cows' -- High Profit, Low Volume
        WHEN profit_quartile_region < 3 AND sales_quartile_region >= 3 THEN 'Question Marks' -- Low Profit, High Volume
        WHEN profit_quartile_region < 3 AND sales_quartile_region < 3 THEN 'Dogs' -- Low Profit, Low Volume
        ELSE 'Other'
    END AS portfolio_quadrant_region
FROM
    RegionalSalesPercentiles
ORDER BY
    region,
    portfolio_quadrant_region DESC
"""

# Run the regional portfolio analysis query
portfolio_query_region_job = client.query(portfolio_query_region)

# Convert regional portfolio analysis results to DataFrame
portfolio_results_region_df = portfolio_query_region_job.to_dataframe()

# Display the results
print("\n--- Strategic Product Portfolio Analysis by Region ---")
portfolio_results_region_df

# Calculate counts and total contribution per quadrant and region
quadrant_region_summary = portfolio_results_region_df.groupby(['region', 'portfolio_quadrant_region']).agg(
    product_count=('product_name', 'count'),
    total_profit_contribution=('total_profit', 'sum'),
    total_sales_contribution=('total_sales', 'sum')
).reset_index()

# Display the regional quadrant summary
print("\n--- Product Portfolio Quadrant Summary by Region ---")

# Optional: Format summary monetary columns for better readability
monetary_cols_summary_region = ['total_profit_contribution', 'total_sales_contribution']
for col in quadrant_region_summary.columns:
    if col in monetary_cols_summary_region:
        quadrant_region_summary[col] = quadrant_region_summary[col].apply(
            lambda x: f"${x:,.2f}" if pd.notnull(x) else None
        )

quadrant_region_summary


In [ ]:
# prompt: Using dataframe quadrant_region_summary: create a bar chart. NOT stacked. cluster by region. color by category. height by profit.

import altair as alt
# Create a bar chart.
# Cluster bars by 'region'.
# Color the bars by 'portfolio_quadrant_region'.
# Map 'total_profit_contribution' to the height of the bars.
chart = alt.Chart(quadrant_region_summary).mark_bar().encode(
    x=alt.X('portfolio_quadrant_region', axis=None), # Map category to the x-axis, hide axis labels
    y=alt.Y('total_profit_contribution'), # Map profit to the y-axis
    color='portfolio_quadrant_region', # Color the bars by category
    column='region' # Create a separate column for each region
).properties(
    title='Total Profit Contribution by Portfolio Quadrant and Region' # Add a title to the chart
)
chart

In [ ]:
# prompt: Using dataframe: quadrant_region_summary How are Office Supplies, Technology, and Furniture distributed across the four quadrants? Analyze by region and by total.

import pandas as pd
import altair as alt

# Simplified SQL query for regional portfolio analysis
regional_portfolio_query = f"""
WITH ProductMetrics AS (
    SELECT
        region,
        category,
        product_name,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales
    FROM `{FULL_TABLE_ID}`
    WHERE category IN ('Office Supplies', 'Technology', 'Furniture')
    GROUP BY region, category, product_name
),
Quartiles AS (
    SELECT *,
        NTILE(4) OVER (PARTITION BY region ORDER BY total_profit) AS profit_quartile,
        NTILE(4) OVER (PARTITION BY region ORDER BY total_sales) AS sales_quartile
    FROM ProductMetrics
)
SELECT
    region,
    category,
    product_name,
    total_profit,
    total_sales,
    CASE
        WHEN profit_quartile >= 3 AND sales_quartile >= 3 THEN 'Stars'
        WHEN profit_quartile >= 3 AND sales_quartile < 3 THEN 'Cash Cows'
        WHEN profit_quartile < 3 AND sales_quartile >= 3 THEN 'Question Marks'
        ELSE 'Dogs'
    END AS quadrant
FROM Quartiles
"""

# Execute query and get results
results_df = client.query(regional_portfolio_query).to_dataframe()

# Function to format monetary values
def format_money(df, columns):
    """Format specified columns as currency"""
    formatted_df = df.copy()
    for col in columns:
        if col in formatted_df.columns:
            formatted_df[col] = formatted_df[col].apply(
                lambda x: f"${x:,.2f}" if pd.notnull(x) else None
            )
    return formatted_df

# Analysis 1: Summary by Region, Quadrant, and Category
regional_summary = results_df.groupby(['region', 'quadrant', 'category']).agg({
    'product_name': 'count',
    'total_profit': 'sum',
    'total_sales': 'sum'
}).rename(columns={'product_name': 'product_count'}).reset_index()

# Analysis 2: Overall Summary by Quadrant and Category (across all regions)
overall_summary = results_df.groupby(['quadrant', 'category']).agg({
    'product_name': 'count',
    'total_profit': 'sum',
    'total_sales': 'sum'
}).rename(columns={'product_name': 'product_count'}).reset_index()

# Display raw results
print("=== Regional Summary (Raw Numbers) ===")
print(regional_summary)

print("\n=== Overall Summary (Raw Numbers) ===")
print(overall_summary)

# Display formatted results
print("\n=== Regional Summary (Formatted) ===")
print(format_money(regional_summary, ['total_profit', 'total_sales']))

print("\n=== Overall Summary (Formatted) ===")
print(format_money(overall_summary, ['total_profit', 'total_sales']))

# Visualization 1: Product count by quadrant and category
chart1 = alt.Chart(overall_summary).mark_bar().encode(
    x=alt.X('quadrant:O', title='Portfolio Quadrant'),
    y=alt.Y('product_count:Q', title='Number of Products'),
    color=alt.Color('category:N', title='Category'),
    tooltip=['quadrant', 'category', 'product_count', 'total_profit', 'total_sales']
).properties(
    title='Product Distribution by Quadrant and Category',
    width=400,
    height=300
)

# Visualization 2: Regional profit distribution
chart2 = alt.Chart(regional_summary).mark_bar().encode(
    x=alt.X('quadrant:O', title='Quadrant'),
    y=alt.Y('total_profit:Q', title='Total Profit'),
    color=alt.Color('category:N', title='Category'),
    column=alt.Column('region:N', title='Region'),
    tooltip=['region', 'quadrant', 'category', 'product_count', 'total_profit']
).properties(
    title='Profit by Quadrant, Category, and Region',
    width=150,
    height=200
)

# Display charts
chart1.show()
chart2.show()

In [ ]:
# prompt: Create a summary of subcategories, broken down by region. Do not create a matrix classification for this query.

import pandas as pd
# Create a summary of subcategories, broken down by region.
subcategory_region_query = f"""
SELECT
    region,
    category,
    subcategory,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit,
    COUNT(DISTINCT product_name) AS unique_products,
    COUNT(DISTINCT order_id) AS number_of_orders
FROM
    `{FULL_TABLE_ID}`
GROUP BY
    region,
    category,
    subcategory
ORDER BY
    region,
    category,
    total_profit DESC -- Order by profit within each region and category
"""

# Run the subcategory by region query
subcategory_region_job = client.query(subcategory_region_query)

# Convert results to DataFrame
subcategory_region_df = subcategory_region_job.to_dataframe()

# Display the results
print("\n--- Subcategory Summary by Region ---")

# Optional: Format monetary columns for better readability
monetary_cols_sub_region = ['total_sales', 'total_profit']
for col in subcategory_region_df.columns:
    if col in monetary_cols_sub_region:
        subcategory_region_df[col] = subcategory_region_df[col].apply(
            lambda x: f"-${abs(x):,.2f}" if pd.notnull(x) and x < 0
            else f"${x:,.2f}" if pd.notnull(x) else None
        )

subcategory_region_df


In [ ]:
# prompt: Using dataframe subcategory_region_df: create a stacked bar chart to visualize the top 3 subcategories in each region. place all bars on a single axis, labeled by region. bar height should indicate total profit for each subcategory. label the Y axis with currency as well.

import altair as alt

# Filter for the top 3 subcategories in each region based on the 'rank' column
top_subcategories = subcategory_region_df[subcategory_region_df['rank'] <= 5]

# Create the stacked bar chart
chart = alt.Chart(top_subcategories).mark_bar().encode(
    x=alt.X('region:N', title='Region'), # Set x-axis to region
    y=alt.Y('total_profit:Q', title='Total Profit', axis=alt.Axis(format='$,.0f')), # Set y-axis to total_profit and format as currency
    color='subcategory:N' # Stack the bars by subcategory
).properties(
    title='Top 5 Subcategories by Profit per Region' # Add a title to the chart
).properties(
    width=400,  # Increase width to 400 pixels
    height=400  # Optional: also set height
)

# Display the chart
chart